In [1]:
"""
/**
 * Implementation of the asymmetric envy free cake-cutting algorithm on a 1-dimensional cake
 *
 * @author Thomas Gibson
 * @date 30/04/2021
 */
"""


import numpy as np, numpy.random
import random as ran


from Agent import Agent


#This function will generate n values that will sum to exactly 1. 
#This will form an array showing how much an agent values a single atom
#This generating function could be changed to make use of other distributions if preferred
def generatevaluations(numatoms): 
    x = np.random.random(numatoms)
    x /= x.sum()
    return x


#This function will obtain the i atoms the input agent, considers to be of at leats half the value of the cake.
#We will update the agents cutter attribute to reflect that they are the cutter.
def firstcut(agent, atoms):
    agent.cutter = 1
    i = 0
    v = 0
    while v <= 0.5:
        v = v + agent.vals[i]
        i+=1
    piece1 = (0, i-1)
    piece2 = (i, atoms-1)
    return piece1, piece2


#This function will determine an agents value for a piece (vop)
def vop(agent, piece):
    value = 0
    a, b = piece   
    
    #This line is needed as the range function is not inclusive on the top end
    b = b+1
    
    for x in range(a, b):
        value = value + agent.vals[x]
    return value


#This function represents the choose portion of cut and choose
def choose(cutter, chooser, piece1, piece2):
    
    #These 2 if statments will allocate the choosers their desired piece
    if vop(chooser, piece1) > vop(chooser, piece2): 
        chooser.allocation = piece1
        cutter.allocation = piece2
    elif vop(chooser, piece2) > vop(chooser, piece1):
        chooser.allocation = piece2
        cutter.allocation = piece1
    
    #Optional Step to help minimise envy freeness.
    #If the chooser has no preference, we will ask the cutter to select a desired piece
    #This step is only useful as we can't guarentee the cutter agent values both pieces exactly equally.
    #The Cut and Choose Procedure does not require this step if we had infinite atoms.
    elif vop(cutter, piece1) > vop(cutter, piece2):
        chooser.allocation = piece2
        cutter.allocation = piece1
    elif vop(secondchooser, piece2) > vop(cutter, piece1):
        chooser.allocation = piece1
        cutter.allocation = piece2    
        
    #In the rare scenario where both agents consider both piece to be equal in value, we can allocate arbitrarily
    else:
        chooser.allocation = piece1
        cutter.allocation = piece2
    return (cutter.allocation, chooser.allocation)

#This function determines if an allocation is envy free or not
def envyfreecheck(agent1, agent2):
    #This if statement determins if both agents consider their own piece to be greater than the other agents.
    #If this is the case, we can conclude the allocation is envy free and return the value 1
    #If it is not envy free we return 0
    if ((vop(agent1, agent1.allocation) - vop(agent1, agent2.allocation) >= 0) and (vop(agent2, agent2.allocation) - vop(agent2, agent1.allocation) >= 0)):
        return 1
    else:
        return 0


#Function to print the final allocation for each agent and their value of the piece they recieved, and their percieved value of the other agent
def printallocation(agent1, agent2):
    print("Agent " ,agent1.name, "recieved atoms within the range" ,agent1.allocation)
    print("This agent obtained a value of " ,vop(agent1, agent1.allocation), ", and consider the other agent to have obtained", vop(agent1, agent2.allocation))
    print("Agent " ,agent2.name, "recieved atoms within the range" ,agent2.allocation)   
    print("This agent obtained a value of " ,vop(agent2, agent2.allocation), ", and consider the other agent to have obtained", vop(agent2, agent1.allocation))    
    if (envyfreecheck(agent1,agent2) == 1):
        print("This allocation was envy free")
    else:
        print("This allocation was not envy free")


#This functon will simple return a random agent, can be used to randomly select the cutter agent        
def randomagent(agent1, agent2):
    rn = ran.random()
    if rn > 0.5:
        return agent1, agent2
    else:
        return agent2, agent1

    
def quantifyenvygenerated(agent1, agent2):
    if envyfreecheck(agent1, agent2) == 1:
        return 0
    elif (vop(agent1, agent1.allocation) - vop(agent1, agent2.allocation) < 0):
        return vop(agent1, agent1.allocation) - vop(agent1, agent2.allocation)
    else:
        return vop(agent2, agent2.allocation) - vop(agent2, agent1.allocation)

In [2]:
#Determine how many atoms we want
atomsexample = 20

#Generate some agents
agentA = Agent("A", generatevaluations(atomsexample))
agentB = Agent("B", generatevaluations(atomsexample))

#Randomly determine which agent is the cutter and chooser
cutter, chooser = randomagent(agentA, agentB)

#Have the cutter cut the cake
piece1, piece2 = firstcut(cutter, atomsexample)

print(piece1)
print(piece2)
#Have agents select pieces
cutterallocation, chooserallocation = choose(cutter, chooser, piece1, piece2)


#Print out the range of pieces allocated to each agent and how they value the pieces and if the allocation was envy free.
printallocation(cutter, chooser)




(0, 10)
(11, 19)
Agent  A recieved atoms within the range (11, 19)
This agent obtained a value of  0.48634859338044434 , and consider the other agent to have obtained 0.5136514066195556
Agent  B recieved atoms within the range (0, 10)
This agent obtained a value of  0.5375515067818166 , and consider the other agent to have obtained 0.46244849321818354
This allocation was not envy free


In [3]:
#Function that does a single iteration of the asymmetric cut and choose
#with random agents and random cutter and chooser for a given number of agents.
#We return a value, 1 or 0 depending if the allocation was envy free or not
def iteratingasymmetric(atoms):
    agentA = Agent("A", generatevaluations(atoms))
    agentB = Agent("B", generatevaluations(atoms))
    cutter, chooser = randomagent(agentA, agentB)
    piece1, piece2 = firstcut(cutter, atoms)
    cutterallocation, chooserallocation = choose(cutter, chooser, piece1, piece2)
    envygenerated = quantifyenvygenerated(cutter, chooser)
    return (envyfreecheck(cutter, chooser), envygenerated)
    
    

In [4]:
#Iterate through 1000 Asymmetric Cut and Choose Procedures and determine number of envy free iterations with 100 atoms.
#Here you can edit the number of iterations or atoms
iterations = 1000
numberofatoms = 100
counter = 0
envygen = 0

for m in range(1, iterations+1, 1):
    ef, envygend = iteratingasymmetric(numberofatoms)
    counter = counter + ef
    envygen = envygen + envygend
percentage = round(((counter/iterations)*100), 2)
averageenvygend = abs(envygen/iterations)

print("With", numberofatoms, "atoms and", iterations, "iterations, we get that a total of",counter,"(",percentage,"%)iterations are envy free.")
print("The average amount of envy generated in any given iteration is of value",averageenvygend)

With 100 atoms and 1000 iterations, we get that a total of 448 ( 44.8 %)iterations are envy free.
The average amount of envy generated in any given iteration is of value 0.007485609223887894
